In [1]:
from gurobipy import *
from math import *

In [2]:

"""

Klaučo, M., Blažek, S., Kvasnica, M., & Fikar, M. (2014, June). 
Mixed-integer SOCP formulation of the path planning problem for heterogeneous multi-vehicle systems. 
In 2014 European Control Conference (ECC) (pp. 1474-1479). IEEE.

https://sci-hub.tw/https://doi.org/10.1109/ECC.2014.6862400


"""


q=((0,0),(0,40),(0,50),(1,49),(46,50),(48,48),(46,46),(50,48),(50,35),(30,5),(30,0));
#note:q9 in the tableI is wrong

n = 10


V_h=90
V_c=18

r=0.35

qs=(0,0)
qf=(50,0)

In [3]:
# Create a new model
m = Model("MISOCP_2014")


# Create variables
Tx = {};
Ty = {};
lx = {};
ly = {};
alpha = {};
f = {};
s = {};


#for SOCP
Cx= {};
Cy= {};
C= {};

d= {};

for i in range(1, n+1):
    Tx[i] = m.addVar(vtype = 'C',name='Tx[%s]'%i);
    Ty[i] = m.addVar(vtype = 'C',name='Ty[%s]'%i);
    lx[i] = m.addVar(vtype = 'C',name='lx[%s]'%i);
    ly[i] = m.addVar(vtype = 'C',name='ly[%s]'%i);
    for j in range(i, n+1):
        alpha[i,j] = m.addVar(lb=0,vtype = 'B',name='alpha[%s,%s]'%(i,j));
        f[i,j] = m.addVar(lb=0,vtype = 'C',name='f[%s,%s]'%(i,j));
        s[i,j] = m.addVar(lb=0,vtype = 'C',name='s[%s,%s]'%(i,j));
        d[i,j]=quicksum(sqrt( (q[k][0]-q[k+1][0])*(q[k][0]-q[k+1][0]) + (q[k][1]-q[k+1][1])*(q[k][1]-q[k+1][1]) ) for k in range(i,j));
        


            


        
# Integrate new variables

m.update()




Academic license - for non-commercial use only


In [4]:

# Add constraint: 
M =200
ctr=0;

for i in range(1, n+1):
    for j in range(i,n+1):
        
        #C18
        m.addConstr( f[i,j] - r <= M*(1-alpha[i,j])  ,'C.18.[%s,%s]'%(i,j));
        
        
        
        #C19a
        
        
        ctr=ctr+1;
    
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19a.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19a.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19a.[%s,%s]'%(i,j));
    
    
        m.addConstr( Cx[ctr]  == Tx[i]-lx[j], 'C19ax.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == Ty[i]-ly[j], 'C19ay.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19a.2.[%s,%s]'%(i,j));
        m.addConstr( C[ctr]-(V_c*f[i,j])  == M*(1-alpha[i,j]), 'C19a.3.[%s,%s]'%(i,j));
        
    
        
        #C19b
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19b.1.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19b.1.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19b.1.[%s,%s]'%(i,j));
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19b.2.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19b.2.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C.C19b.2.[%s,%s]'%(i,j));
        
        
        m.update()
        m.addConstr( Cx[ctr-1]  == Tx[i]-q[i][0], 'C19bx.1.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr-1]  == Ty[i]-q[i][1], 'C19by.1.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr-1]*Cx[ctr-1] + Cy[ctr-1]*Cy[ctr-1] <=  C[ctr-1]*C[ctr-1], 'C19b.2.1.[%s,%s]'%(i,j));
        
        
        m.addConstr( Cx[ctr]  == lx[j]-q[j][0], 'C19bx.1.2.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == ly[j]-q[j][1], 'C19by.1.2.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19b.2.2.[%s,%s]'%(i,j));
        
       

       
        
        m.addConstr( C[ctr] + d[i,j]+C[ctr-1]-V_h*f[i,j] == M*(1-alpha[i,j]), 'C.19b.3.[%s,%s]'%(i,j));
        
        
     
        
        
for i in range(1, n+1):
    for j in range(i,n):

        #C19c
      
        
        ctr=ctr+1;
        
        Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cx.C19c.[%s,%s]'%(i,j));
        Cy[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='Cy.C19c.[%s,%s]'%(i,j));
        C[ctr] = m.addVar(lb=0.0,vtype = 'C',name='C19c.[%s,%s]'%(i,j));
            
        m.addConstr( Cx[ctr]  == lx[j]-Tx[j+1], 'C19cx.1.[%s,%s]'%(i,j)); 
        m.addConstr( Cy[ctr]  == ly[j]-Ty[j+1], 'C19cy.1.[%s,%s]'%(i,j));
        m.addQConstr( Cx[ctr]*Cx[ctr] + Cy[ctr]*Cy[ctr] <=  C[ctr]*C[ctr], 'C19c.2.[%s,%s]'%(i,j));
        m.addConstr( C[ctr]-(V_c*s[i,j])  == M*(1-alpha[i,j]), 'C19c.3.[%s,%s]'%(i,j));
        
        

                 

#C2
for k in range(1, n+1):
    m.addConstr( quicksum(quicksum(alpha[i,j] for j in range(k,n+1)) for i in range(1,k+1))  == 1, 'C2.[%s]'%k); 
        
        


        



In [5]:
###Objective Function
a = m.addVar(lb=0,vtype = 'C',name='a');
b = m.addVar(lb=0,vtype = 'C',name='b');

ctr=ctr+1;
Cx[ctr] = m.addVar(lb=-GRB.INFINITY,vtype = 'C',name='nnn');


m.addConstr( Cx[ctr] == lx[n]-qf[0], 'obj_bx');



m.addQConstr( Tx[1]*Tx[1]+Ty[1]*Ty[1] <= a*a, 'obj_a');
m.addQConstr( Cx[ctr]*Cx[ctr] + ly[n]*ly[n] <= b*b, 'obj_b');


obj = (a+b)/V_c + quicksum(quicksum(f[i,j] for j in range(i,n+1)) for i in range(1,n+1) ) + quicksum(quicksum(s[i,j] for j in range(i,n)) for i in range(1,n+1)) 
m.setObjective(obj,GRB.MINIMIZE)

In [6]:
m.optimize()

Optimize a model with 641 rows, 838 columns and 1892 nonzeros
Model has 212 quadratic constraints
Variable types: 783 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 369 rows and 401 columns
Presolve time: 0.03s
Presolved: 272 rows, 437 columns, 804 nonzeros
Variable types: 415 continuous, 22 integer (22 binary)

Root relaxation: objective 0.000000e+00, 48 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8          -    0.00000      -     -    0s
     0     0    0.00000    0   11          -    0.00000      -     -    0s
H    0     0                       6.4162449    0.00000   100%     -    0s
*    0     0               0       6.4162

In [7]:

obj = m.getObjective()
print("t_m:",obj.getValue(),"hours\n")
print ('runtime is',m.Runtime,"secs\n")
print("M:",M,"\n")
ctr=0
for i in range(1,n+1):
    for j in range(i,n+1):
        if alpha[i,j].X >0.1:
            ctr=ctr+1
            print("Flyover no.", ctr, "from q",i, " to q",j,":" )
            for k in range(i,j+1):
                print("q", k," : (",q[k][0],", ", q[k][1],")")
            print("\nT", i," : (",Tx[i].X,", ", Ty[i].X,")")
            print("l", j," : (",lx[j].X,", ", ly[j].X,")\n-----")

t_m: 6.248034955293704 hours

runtime is 1.6141738891601562 secs

M: 200 

Flyover no. 1 from q 1  to q 1 :
q 1  : ( 0 ,  40 )

T 1  : ( 6.377332214822178 ,  31.113047312798805 )
l 1  : ( 7.329914430409178 ,  35.88779513813707 )
-----
Flyover no. 2 from q 2  to q 3 :
q 2  : ( 0 ,  50 )
q 3  : ( 1 ,  49 )

T 2  : ( 7.365485119695255 ,  36.06753834699945 )
l 3  : ( 12.216277579555681 ,  40.08745261199131 )
-----
Flyover no. 3 from q 4  to q 6 :
q 4  : ( 46 ,  50 )
q 5  : ( 48 ,  48 )
q 6  : ( 46 ,  46 )

T 4  : ( 33.206596764892154 ,  40.33671006336561 )
l 6  : ( 39.32010144325413 ,  38.815180504252986 )
-----
Flyover no. 4 from q 7  to q 7 :
q 7  : ( 50 ,  48 )

T 7  : ( 39.32010204079527 ,  38.81518006718392 )
l 7  : ( 41.38865542349201 ,  32.86446088451957 )
-----
Flyover no. 5 from q 8  to q 8 :
q 8  : ( 50 ,  35 )

T 8  : ( 41.42709573623922 ,  32.00640830565824 )
l 8  : ( 41.63134422934149 ,  27.490066414563795 )
-----
Flyover no. 6 from q 9  to q 9 :
q 9  : ( 30 ,  5 )

T 9  : ( 4

In [8]:
#m.getVars()

In [9]:
#f

In [10]:
#alpha

In [11]:
#m.write("out.lp")

In [12]:
#m.computeIIS()

In [ ]:
status = m.status
if status == GRB.Status.OPTIMAL:
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    print('Obj: %g' % m.objVal)
       
elif status == GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    # do IIS
    m.computeIIS()
    for c in m.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)

In [13]:
#m.write("SOCP_2014.lp")

In [14]:
#m.computeIIS()